In [1]:
!git clone https://github.com/robertocotesta/neural_covariance.git

Cloning into 'neural_covariance'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 20 (delta 2), reused 8 (delta 0), pack-reused 0
Unpacking objects: 100% (20/20), done.


In [2]:
import pickle
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch import nn
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

## Data wrangling

In [3]:
param_list = ['Mc', 'eta', 'DL', 'tc', 'phic', 'iota', 'ra', 'dec', 'psi']

In [4]:
def recreate_symmetric_matrix(size_X,X_flatten):
    X = np.zeros((size_X,size_X))
    X[np.triu_indices(X.shape[0], k = 0)] = X_flatten
    X = X + X.T - np.diag(np.diag(X))
    return X

def flatten_symmetric_matrix(X):
    return X[np.triu_indices(X.shape[0], k = 0)]

In [5]:
with open('/content/neural_covariance/Data/input.pkl','rb') as f:
    input_dataset = pickle.load(f)
with open('/content/neural_covariance/Data/output.pkl','rb') as f:
    output_dataset = pickle.load(f)    
with open('/content/neural_covariance/Data/SNRs.pkl','rb') as f:
    SNR_dataset = pickle.load(f)      

In [6]:
input_df = pd.DataFrame.from_dict(input_dataset, orient='index',columns=param_list)

In [7]:
# Each matrix is symmetric, hence I only extract and flatten its triangular upper part
output_df = pd.DataFrame(data=[flatten_symmetric_matrix(output_dataset[i]) for i in output_dataset.keys()])

In [8]:
# Load SNR for interpolation
SNR_df = pd.DataFrame(data=SNR_dataset)

In [9]:
sc_Input = StandardScaler()
sc_Output = StandardScaler()
input_df = input_df[['Mc', 'eta', 'DL', 'iota', 'ra', 'dec']]
# Scaling input
scaled_input_df = sc_Input.fit_transform(input_df)
# Scaling output
scaled_output_df = sc_Output.fit_transform(output_df)

In [10]:
# Split test, train, CV for Fisher
# X_train, X_test, y_train, y_test = train_test_split(scaled_input_df,output_df.values,test_size=0.4,random_state=0)
# X_CV, X_test, y_CV, y_test = train_test_split(X_test,y_test,test_size=0.5,random_state=0)

In [142]:
# Split test, train, CV for SNR
X_train, X_test, y_train, y_test = train_test_split(scaled_input_df,SNR_df.values,test_size=0.4,random_state=0)
X_CV, X_test, y_CV, y_test = train_test_split(X_test,y_test,test_size=0.5,random_state=0)

In [77]:
# i_max = 2000
# X_train, y_train = X_train[:i_max], y_train[:i_max]

## Build interpolant

In [78]:
# Definition of the device for training
device = "cuda" if torch.cuda.is_available() else "cpu"

In [79]:
# General settings for the training
epochs = 100000
lr = 0.001
batch_size  = y_train.shape[0]
input_layer_dim = X_train.shape[1]
output_layer_dim = y_train.shape[1]

In [143]:
# Definition of the neural network
class interpolant(nn.Module):
    def __init__(self,dim_input,dim_output):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(dim_input, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, dim_output),
        )
        
    def forward(self,X):
        return self.linear_relu_stack(X)

In [144]:
# Definition of dataset for interpolation
class DatasetInterpolation(Dataset):
    def __init__(self,X_values,y_values):
        self.X_values = torch.from_numpy(X_values).float()
        self.y_values = torch.from_numpy(y_values).float()

    def __len__(self):
        return len(self.y_values)

    def __getitem__(self, idx):
        return self.X_values[idx], self.y_values[idx]

In [145]:
model = interpolant(input_layer_dim,output_layer_dim).to(device)

In [146]:
training_data = DatasetInterpolation(X_train,y_train)
test_data = DatasetInterpolation(X_CV,y_CV)

In [147]:
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True,pin_memory=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True,pin_memory=True)

In [148]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:.2E}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

    test_loss /= num_batches
    print(f"Avg loss: {test_loss:>8f} \n")

In [149]:
# Definition of loss function and optimizer
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),weight_decay=10)

# Training loop
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 5.06E+05  [    0/ 6917]
Avg loss: 554533.618750 

Epoch 2
-------------------------------
loss: 4.83E+05  [    0/ 6917]
Avg loss: 556300.956250 

Epoch 3
-------------------------------
loss: 5.60E+05  [    0/ 6917]
Avg loss: 541977.037500 

Epoch 4
-------------------------------
loss: 4.85E+05  [    0/ 6917]
Avg loss: 538679.643750 

Epoch 5
-------------------------------
loss: 5.42E+05  [    0/ 6917]
Avg loss: 505675.068750 

Epoch 6
-------------------------------
loss: 5.30E+05  [    0/ 6917]
Avg loss: 460634.200000 

Epoch 7
-------------------------------
loss: 4.58E+05  [    0/ 6917]
Avg loss: 402642.681250 

Epoch 8
-------------------------------
loss: 3.95E+05  [    0/ 6917]
Avg loss: 312048.681250 

Epoch 9
-------------------------------
loss: 2.15E+05  [    0/ 6917]
Avg loss: 254895.150000 

Epoch 10
-------------------------------
loss: 2.65E+05  [    0/ 6917]
Avg loss: 233109.834375 

Epoch 11
------------------------------

KeyboardInterrupt: ignored

In [ ]:
i = 250
model(torch.from_numpy(X_CV[i]).float().to(device)), y_CV[i]

In [1]:
i = 250
model(torch.from_numpy(X_train[i]).float().to(device)), y_train[i]

NameError: ignored